In [1]:
import os

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D  # noqa
from scipy.linalg import svd

import mne
from mne.preprocessing import (
    ICA,
    compute_proj_ecg,
    compute_proj_eog,
    create_ecg_epochs,
    create_eog_epochs,
)


def setup_3d_axes():
    ax = plt.axes(projection="3d")
    ax.view_init(azim=-105, elev=20)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.set_xlim(-1, 5)
    ax.set_ylim(-1, 5)
    ax.set_zlim(0, 5)
    return ax

In [2]:
eeg = mne.io.read_raw_brainvision('EEGs\sub-010002.vhdr', eog=('VEOG',), preload=True, misc='auto', scale=1, verbose=None)

Extracting parameters from EEGs\sub-010002.vhdr...
Setting channel info structure...
Reading 0 ... 2554999  =      0.000 ...  1022.000 secs...


In [4]:
def filtering(eeg, lf=1, hf=20):

    eeg_filtered = eeg.copy().filter(l_freq=lf, h_freq=hf, fir_design="firwin", verbose=False)
    
    return eeg_filtered

In [13]:
eeg_filtered = filtering(eeg)

In [9]:
def oc_artifacts(eeg, lf=1, hf=20, nc=7, t=1):

    
    # STEP 1: Compute epochs from ocular artifacts
    eog_epochs = create_eog_epochs(eeg, ch_name='VEOG', l_freq=lf, h_freq=hf, baseline=(-0.5, -0.2))
    
    # STEP 2: Train ica object, dividing in components
    ica = ICA(n_components=nc, random_state=20)
    ica.fit(eeg)

    # STEP 3: Compute the indices which compose the ocular artefacts 
    eog_inds, _ = ica.find_bads_eog(eog_epochs, threshold=t, l_freq=lf, h_freq=hf)
    # In case no component is recognized as artifact we decrease the threshold as we know that each signal will have ocular artifacts
    while len(eog_inds) == 0:
        eog_inds, _ = ica.find_bads_eog(eog_epochs, threshold=t-0.1, l_freq=lf, h_freq=hf)

    # STEP 4: Remove the components related with the EOG from the raw data
    ica.exclude = eog_inds
    cleaned_eeg = eeg.copy()
    ica.apply(cleaned_eeg)

    # STEP 5: return the cleaned eeg
    return cleaned_eeg

In [14]:
cleaned_eeg = oc_artifacts(eeg_filtered)

Using EOG channel: VEOG
EOG channel index for this subject is: [16]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel VEOG for blink detection
Setting up band-pass filter from 1 - 20 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 20.25 Hz)
- Filter length: 25000 samples (10.000 s)

Now detecting blinks and generating corresponding events


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Found 62 significant peaks
Number of EOG events detected: 62
Not setting metadata
62 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 62 events and 2501 original time points ...
0 bad epochs dropped
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 7 components
Fitting ICA took 37.4s.
Using EOG channel: VEOG
Applying ICA to Raw instance
    Transforming to ICA space (7 components)
    Zeroing out 1 ICA component
    Projecting back using 61 PCA components


In [26]:
def dictionary(eeg, events):

    sf = eeg.info['sfreq']

    d = {}
    
    c1 = 0
    c2 = 0
    c3 = 0
    
    list_events = []
    
    for i, item in enumerate(events):
        #print('Event', i)
        
        if i != 0 and item[2] != events[i-1][2]:
            #print('Cambio:', item[2], events[i-1][2])
            
            prev_event = events[i-1][2]
            if prev_event == 99999 or prev_event == 10001:
                pass
            
            elif prev_event == 1:
                d['Dc' + str(c1 + 1)] = {
                    #'events': list_events,
                    'start_sample': start_time,
                    'end_sample': item[0],
                    'start_time': start_time / sf,
                    'end_time': item[0] / sf,  # Tiempo de end es el tiempo actual
                    'time': (item[0] - start_time) / sf
                }
                c1 += 1
                
            elif prev_event == 200:
                d['Oe' + str(c2 + 1)] = {
                    #'events': list_events,
                    'start_sample': start_time,
                    'end_sample': item[0],
                    'start_time': start_time / sf,
                    'end_time': item[0] / sf,  # Tiempo de end es el tiempo actual
                    'time': (item[0] - start_time) / sf
                }
                c2 += 1
                
            elif prev_event == 210:
                d['Ce' + str(c3 + 1)] = {
                    #'events': list_events,
                    'start_sample': start_time,
                    'end_sample': item[0],
                    'start_time': start_time / sf,
                    'end_time': item[0] / sf,  # Tiempo de end es el tiempo actual
                    'time': (item[0] - start_time) / sf
                }
                c3 += 1
            
            # Reiniciar la lista de eventos y establecer el tiempo de inicio para el nuevo evento
            list_events = []
            start_time = item[0]
        
        # Añadir evento a la lista de eventos
        list_events.append(item)
    
    # Guardar los datos del último evento
    last_event = events[-1][2]
    if last_event == 99999 or last_event == 10001:
        pass
    elif last_event == 1:
        d['Dc' + str(c1 + 1)] = {
            #'events': list_events,
            'start_sample': start_time,
            'end_sample': item[0],
            'start_time': start_time / sf,
            'end_time': events[-1][0] / sf,  # Tiempo de end es el tiempo del último evento
            'time': (events[-1][0] - start_time) / sf
        }
    elif last_event == 200:
        d['Oe' + str(c2 + 1)] = {
            #'events': list_events,
            'start_sample': start_time,
            'end_sample': item[0],
            'start_time': start_time / sf,
            'end_time': events[-1][0] / sf,  # Tiempo de end es el tiempo del último evento
            'time': (events[-1][0] - start_time) / sf
        }
    elif last_event == 210:
        d['Ce' + str(c3 + 1)] = {
            #'events': list_events,
            'start_sample': start_time,
            'end_sample': item[0],
            'start_time': start_time / sf,
            'end_time': events[-1][0] / sf,  # Tiempo de end es el tiempo del último evento
            'time': (events[-1][0] - start_time) / sf
        }

    return d

In [27]:
events, event_dict = mne.events_from_annotations(eeg)
d = dictionary(cleaned_eeg, events)

Used Annotations descriptions: ['Comment/no USB Connection to actiCAP', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S200', 'Stimulus/S210']


In [28]:
d

{'Dc1': {'start_sample': 9979,
  'end_sample': 15962,
  'start_time': 3.9916,
  'end_time': 6.3848,
  'time': 2.3932},
 'Ce1': {'start_sample': 15962,
  'end_sample': 171846,
  'start_time': 6.3848,
  'end_time': 68.7384,
  'time': 62.3536},
 'Dc2': {'start_sample': 171846,
  'end_sample': 171871,
  'start_time': 68.7384,
  'end_time': 68.7484,
  'time': 0.01},
 'Oe1': {'start_sample': 171871,
  'end_sample': 326154,
  'start_time': 68.7484,
  'end_time': 130.4616,
  'time': 61.7132},
 'Dc3': {'start_sample': 326154,
  'end_sample': 326178,
  'start_time': 130.4616,
  'end_time': 130.4712,
  'time': 0.0096},
 'Ce2': {'start_sample': 326178,
  'end_sample': 481053,
  'start_time': 130.4712,
  'end_time': 192.4212,
  'time': 61.95},
 'Dc4': {'start_sample': 481053,
  'end_sample': 481075,
  'start_time': 192.4212,
  'end_time': 192.43,
  'time': 0.0088},
 'Oe2': {'start_sample': 481075,
  'end_sample': 635896,
  'start_time': 192.43,
  'end_time': 254.3584,
  'time': 61.9284},
 'Dc5': {'

In [31]:
def segment_times(d):
    
    st_ce = []
    st_oe = []
    
    for key, value in d.items():
        if key.startswith('Ce'):
            st_ce.append(value['start_time'])
        elif key.startswith('Oe'):
            st_oe.append(value['start_time'])

    return st_ce, st_oe

In [32]:
start_times_ce, start_times_oe = segment_times(d)

In [35]:
def segment_samples(d):

    ss_ce = []
    ss_oe = []
    es_ce = []
    es_oe = []
    
    for key, value in d.items():
        if key.startswith('Ce'):
            ss_ce.append(value['start_sample'])
        elif key.startswith('Oe'):
            ss_oe.append(value['start_sample'])
    
    for key, value in d.items():
        if key.startswith('Ce'):
            es_ce.append(value['end_sample'])
        elif key.startswith('Oe'):
            es_oe.append(value['end_sample'])

    return ss_ce, ss_oe, es_ce, es_oe

In [36]:
start_samples_ce, start_samples_oe, end_samples_ce, end_samples_oe = segment_samples(d)

In [37]:
print("Start times de eventos EYES CLOSED:", start_times_ce)
print("Start times de eventos EYES OPENED:", start_times_oe)
print("-------------------------------------")
print("Start samples de eventos EYES CLOSED:", start_samples_ce)
print("Start samples de eventos EYES OPENED:", start_samples_oe)
print("-----------------------------------------")
print("End samples de eventos EYES CLOSED:", end_samples_ce)
print("End samples de eventos EYES OPENED:", end_samples_oe)

Start times de eventos EYES CLOSED: [6.3848, 130.4712, 254.368, 378.0268, 501.6644, 626.312, 750.3404, 874.874]
Start times de eventos EYES OPENED: [68.7484, 192.43, 316.242, 439.6264, 564.228, 688.5316, 812.6032, 937.3228]
-------------------------------------
Start samples de eventos EYES CLOSED: [15962, 326178, 635920, 945067, 1254161, 1565780, 1875851, 2187185]
Start samples de eventos EYES OPENED: [171871, 481075, 790605, 1099066, 1410570, 1721329, 2031508, 2343307]
-----------------------------------------
End samples de eventos EYES CLOSED: [171846, 481053, 790581, 1099042, 1410547, 1721306, 2031485, 2343283]
End samples de eventos EYES OPENED: [326154, 635896, 945043, 1254136, 1565757, 1875828, 2187161, 2488309]


In [69]:
def division_segments(data, ss, es):
    """
    Function to divide EEG data into segments based on given start and end sample indices.

    Parameters:
    - data (numpy.ndarray): EEG data array.
    - ss (list): List of segment start sample indices.
    - es (list): List of segment end sample indices.

    Returns:
    - segments (list): List of EEG data segments.
    """
    
    segments = []

    for start, end in zip(ss, es):
        segment = data[:, start:end+1]
        segments.append(segment)

    return segments


In [39]:
data = eeg._data

segments_ce = division_segments(data, start_samples_ce, end_samples_ce)
segments_oe = division_segments(data, start_samples_oe, end_samples_oe)

print(len(segments_ce))
print(len(segments_oe))

8
8


In [46]:
len(segments_ce[0][0])

155885

In [54]:
def union_segments_df(eeg, s_ce, s_oe):
    """
    Function to combine segments of "Closed Eyes" and "Open Eyes" events into a single list.

    Parameters:
    - eeg (mne.io.Raw): Raw EEG data.
    - s_ce (list): List of segments for "Ce" events.
    - s_oe (list): List of segments for "Oe" events.

    Returns:
    - segments (list): Combined list of segments for "Closed Eyes" and "Open Eyes" events, each converted in a DataFrame.
    """
    
    segments = []

    for i in range(len(s_ce)):
        segments.append(pd.DataFrame(data=np.transpose(s_ce[i]), columns=eeg.ch_names))
        segments.append(pd.DataFrame(data=np.transpose(s_oe[i]), columns=eeg.ch_names))

    return segments

In [51]:
segments = union_segments(cleaned_eeg, segments_ce, segments_oe)

In [52]:
segments[0]

,Fp1,Fp2,F7,F3,Fz,F4,F8,FC5,FC1,FC2,...,TP8,P5,P1,P2,P6,PO7,PO3,POz,PO4,PO8
0,0.000444,0.000433,0.000103,0.000114,0.000069,0.000120,0.000069,0.000016,0.000024,0.000037,...,-0.000064,-0.000079,-0.000081,-0.000078,-0.000104,-0.000162,-0.000106,-0.000088,-0.000112,-0.000121
1,0.000449,0.000443,0.000096,0.000110,0.000070,0.000121,0.000072,0.000003,0.000021,0.000036,...,-0.000063,-0.000074,-0.000079,-0.000075,-0.000100,-0.000157,-0.000107,-0.000084,-0.000111,-0.000120
2,0.000452,0.000445,0.000096,0.000104,0.000068,0.000116,0.000067,-0.000004,0.000016,0.000033,...,-0.000064,-0.000078,-0.000081,-0.000076,-0.000100,-0.000158,-0.000111,-0.000086,-0.000107,-0.000117
3,0.000440,0.000446,0.000095,0.000109,0.000067,0.000115,0.000066,-0.000002,0.000019,0.000034,...,-0.000065,-0.000082,-0.000082,-0.000072,-0.000100,-0.000157,-0.000110,-0.000087,-0.000106,-0.000112
4,0.000425,0.000442,0.000097,0.000111,0.000063,0.000114,0.000065,0.000003,0.000018,0.000031,...,-0.000064,-0.000085,-0.000082,-0.000076,-0.000102,-0.000160,-0.000111,-0.000089,-0.000105,-0.000113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155880,-0.000146,-0.000156,0.000014,0.000009,-0.000049,-0.000022,-0.000065,0.000077,0.000037,0.000026,...,0.000130,0.000060,0.000047,0.000048,0.000065,0.000052,0.000049,0.000067,0.000053,0.000106
155881,-0.000150,-0.000161,0.000021,0.000008,-0.000053,-0.000029,-0.000078,0.000082,0.000035,0.000023,...,0.000122,0.000062,0.000045,0.000045,0.000061,0.000056,0.000049,0.000063,0.000048,0.000103
155882,-0.000149,-0.000164,0.000031,0.000014,-0.000052,-0.000025,-0.000062,0.000090,0.000037,0.000023,...,0.000122,0.000067,0.000046,0.000046,0.000063,0.000057,0.000051,0.000064,0.000047,0.000104
155883,-0.000151,-0.000160,0.000032,0.000014,-0.000051,-0.000023,-0.000056,0.000090,0.000039,0.000027,...,0.000125,0.000070,0.000051,0.000046,0.000068,0.000057,0.000053,0.000063,0.000050,0.000107
